# Create FloodAdapt site.toml

This notebook creates a baseline site.toml configuration file for FloodAdapt, based in part on the SFINCS and Delft-FIAT models set up with the previous notebooks.

Missing entries that are not optional in the SiteModel class will throw errors when trying to read in the site.toml. Optional entries that are missing will be set to None during read-in.

In [1]:
from pathlib import Path
import numpy as np
#import hydromt
from hydromt_sfincs import SfincsModel
from hydromt_fiat.fiat import FiatModel
from pprint import pprint
import shutil
from os import makedirs
import tomli_w

import sys
sys.path.insert(0, 'c:/Repositories/FloodAdapt/')

from flood_adapt.object_model.site import Site

## Set paths and read models
First set the paths to the overall FloodAdapt folder, and the SFINCS and FIAT models used, and finally read those in.

Also here we set the path to where we want to save the site.toml file to.

In [2]:
model_name = 'Humber'
model_path = Path('c:/Repositories/DT-flood/FloodAdapt_database') / model_name

sf_root = model_path / Path("static/templates/overland")
fiat_root = model_path / Path("static/templates/fiat")

site_fn = model_path / "static" / "site" / "site.toml"

sf = SfincsModel(root=sf_root,mode="r")
sf.read()

fiatmodel = FiatModel(root=fiat_root,mode="r")
fiatmodel.read()

Empty dictionary to store all the site.toml settings in before writing it to a file.

In [3]:
site_dict = {}

## Base naming
Set some base info about the model, name, description. Latitude and longitude we take from the SFINCS domain

In [4]:
base_dict = {
    "name": model_name,
    "description": "Humber Delta, UK",
    "lat": round(sf.region.to_crs(4326).unary_union.centroid.y,2),
    "lon": round(sf.region.to_crs(4326).unary_union.centroid.x,2)
}

site_dict.update(base_dict)
pprint(site_dict)

{'description': 'Humber Delta, UK',
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber'}


## SFINCS settings

Here we specify some SFINCS settings/properties:
- `csname`: Name of the SFINCS CRS
- `cstype`: CRS type, one of 'projected','spherical'
- `version`: SFINCS release version used
- `offshore_model`: Name of the offshore model, empty string for no model
- `overland_model`: Name of overland model
- `datum_offshore_model`: Reference DEM datum for offshore model
- `datum_overland_model`: Reference DEM datum for overland model
- `diff_datum_offshore_overland`: height difference between offshore and overland reference (a UnitfulValue dict)
- `tidal_components`: Specification of tides used (a filepath)
- `ambient_air_pressure`: average air pressure, taken from SFINCS forcing
- `floodmap_no_data_value`: ID for missing data in floodmap
- `floodmap_units`: unit of waterlevel/depth in floodmap (fully written out, FloodAdapt cannot parse abbreviations)

None of the sfincs entry and the above options/entries into the site.toml are optional. A value has to be entered even when the option is not used (e.g. an empty string for offshore_model when no offshore_model is used)



In [5]:
# floodmap entries can be automated once sfincs has run once, but to run sfincs in floodadapt context site.toml needs to exist first

sfincs_dict = {
    "csname": sf.crs.name,
    "cstype": sf.crs.type_name[:9].lower(), # this only works cause projected and spherical (only allowed inputs) have same length coincidentally
    "version": "SFINCS_v2.0.1_tag_exe", #"sfincs20_AlpeDHuez_release",
    "offshore_model": "",
    "overland_model": Path(sf.root).stem,
    "ambient_air_pressure": round(sf.forcing["press_2d"].values.mean()),
    "floodmap_units": "meters",
    "save_simulation": False,
}

site_dict.update({"sfincs": sfincs_dict})
pprint(site_dict)

{'description': 'Humber Delta, UK',
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'offshore_model': '',
            'overland_model': 'overland',
            'save_simulation': False,
            'version': 'SFINCS_v2.0.1_tag_exe'}}


## Water level

Here we specify how the different data used for topography, bathymetry relate to each other

In [6]:
wl_dict = {
    "reference": {
        "name": "MSL",
        "height": {
            "value": 0,
            "units": "meters"
        }
    },
    "localdatum": {
        "name": "EGM2008",
        "height": {
            "value": 0,
            "units": "meters"
        }
    },
    "msl": {
        "name": "MSL",
        "height": {
            "value": 0,
            "units": "meters"
        }
    },
    }

site_dict.update({"water_level": wl_dict})
pprint(site_dict)

{'description': 'Humber Delta, UK',
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'offshore_model': '',
            'overland_model': 'overland',
            'save_simulation': False,
            'version': 'SFINCS_v2.0.1_tag_exe'},
 'water_level': {'localdatum': {'height': {'units': 'meters', 'value': 0},
                                'name': 'EGM2008'},
                 'msl': {'height': {'units': 'meters', 'value': 0},
                         'name': 'MSL'},
                 'reference': {'height': {'units': 'meters', 'value': 0},
                               'name': 'MSL'}}}


## FIAT settings

Here we specify some FIAT setting/properties, and create the static/bfe folder if one doesn't exist yet:
- `exposure_crs`: CRS of the exposure file, taken from the FIAT model
- `building_foortprints`: Specification of building footprints in FIAT domain (filepath to shapefile, OPTIONAL)
- `non_building_names`: ID for non-building objects (OPTIONAL)
- `aggregation`: different aggregation levels to use (dict containing name, filepath, and relevant field name)
- `bfe`: Base Flood Elevation height reference (dict containing geom, field name, and (optional) table, US only)

The building_footprints, non_building_names, and bfe.table are optional, meaning they will be set to None when the site.toml is read in.

In [7]:
if not (model_path / "static" / "bfe").exists():
    shutil.copytree((model_path.parents[1] / "data" / "base_flood_elevation"),(model_path / "static" / "bfe"))

if not (fiat_root/"aggregation_areas").exists():
    makedirs(fiat_root/"aggregation_areas")

agg_area_folder = Path('c:/Repositories/DT-flood/Data/aggregation_zones')
aggregation_area_fn = agg_area_folder/"gadm41_GBR_3.shp"
target_file = fiat_root / "aggregation_areas" / aggregation_area_fn.name
if not target_file.exists():
    target_file.touch()
shutil.copy(aggregation_area_fn,target_file)

fiat_dict = {
    "exposure_crs": fiatmodel.exposure.crs,
    "floodmap_type": "water_level", # This depends on sfincs is subgrid or not? Automate from sfincs results?
    # "building_footprints": "",
    # "non_building_names": [],
    "aggregation": [{
        "name": "gadm_level3",
        "file": "../templates/fiat/aggregation_areas/"+target_file.name,
        "field_name": "ADM_ADM_3",
    }],
    "bfe": {
        "geom": "",
        "field_name": "",
        # "table": ""
        }
}

site_dict.update({"fiat": fiat_dict})
pprint(site_dict)

{'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'offshore_model': '',
            'overland_model': 'overland',
            'save_simulation': False,
            'version': 'SFINCS_v2.0.1_tag_exe'},
 'water_level': {'localdatum': {'height': {'units': 'meters', 'value': 0},
                                'name': 'EGM2008'},
                 'msl': {'height': {'units': 'meters', 'value': 0},
                         'name': 'MSL'},
                 'reference': {'height': {'units': 'meters', 'value': 0},
                               'name': 'MSL'}}}


## Cyclone Tracking Database

File path to the cyclone database. This is not optional, use empty string when no cyclones are modeled.

In [8]:
if not (model_path / "static" / "cyclone_track_database").exists():
    shutil.copytree((model_path.parents[1] / "data" / "cyclone_database"),(model_path / "static" / "cyclone_track_database"))

cdt_dict = {
    "file": ""
}

site_dict.update({"cyclone_track_database": cdt_dict})
pprint(site_dict)

{'cyclone_track_database': {'file': ''},
 'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'offshore_model': '',
            'overland_model': 'overland',
            'save_simulation': False,
            'version': 'SFINCS_v2.0.1_tag_exe'},
 'water_level': {'localdatum': {'height': {'units': 'meters', 'value': 0},
                                'name': 'EGM2008'},
                 'msl': {'height': {'units': 'meters', 'value': 0},
                         'name': 'MSL'},
                 'reference': {'height': {'units': 'meters', 'value': 0},
                               'n

## Sea level rise

Details of the included sea level rise:
- `relative_to_year`: which year is used as a reference point.
- `vertical_offset`: sea level increase or decrease relative to specified year (UnitfulLength, a dict)

The /static/slr folder is created when one doesn't exist yet

In [9]:
if not (model_path / "static" / "slr").exists():
    shutil.copytree((model_path.parents[1] / "data" / "sea_level_projections"),(model_path / "static" / "slr"))

slr_dict = {
    "relative_to_year": 2020,
    "vertical_offset": {
        "value": 0.2,
        "units": "meters"
    }
}

site_dict.update({"slr": slr_dict})
pprint(site_dict)

{'cyclone_track_database': {'file': ''},
 'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'offshore_model': '',
            'overland_model': 'overland',
            'save_simulation': False,
            'version': 'SFINCS_v2.0.1_tag_exe'},
 'slr': {'relative_to_year': 2020,
         'vertical_offset': {'units': 'meters', 'value': 0.2}},
 'water_level': {'localdatum': {'height': {'units': 'meters', 'value': 0},
                                'name': 'EGM2008'},
                 'msl': {'height': {'units': 'meters', 'value': 0},
                         'name': 'MSL'},
        

## GUI

Some settings for the GUI units. This is not optional. (Double check that nans for tide amplitude actually works)

In [10]:
gui_dict = {
    "tide_harmonic_amplitude": {
        "value": 0,
        "units": "meters"
    },
    "default_length_units": "meters",
    "default_distance_units": "meters",
    "default_area_units": "m2",
    "default_volume_units": "m3",
    "default_velocity_units": "m/s",
    "default_discharge_units": "m3/s",
    "default_intensity_units": "mm/hr",
    "default_direction_units": "deg N",
    "default_cumulative_units": "meters",
}
site_dict.update({"gui": gui_dict})
pprint(site_dict)

{'cyclone_track_database': {'file': ''},
 'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'gui': {'default_area_units': 'm2',
         'default_discharge_units': 'm3/s',
         'default_distance_units': 'meters',
         'default_intensity_units': 'mm/hr',
         'default_length_units': 'meters',
         'default_velocity_units': 'm/s',
         'default_volume_units': 'm3',
         'tide_harmonic_amplitude': {'units': 'meters', 'value': 0}},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'offshore_model': '',
            'overland_model': 'overland',
            'save_simulation': False,
            'version': 'SFI

## Risk

Set risk settings/properties (not optional, also not for event_mode = single_event):
- `return_periods`: How often the event reoccurs
- `flooding_threshold`: How high a water level counts as a flood event (UnitfulValue, a dict)

In [11]:
risk_dict = {
    "return_periods": [1, 2, 5, 10, 25, 50, 100],
    "flooding_threshold": {
        "value": 0.15,
        "units": "meters"
    }
}

site_dict.update({"risk": risk_dict})
pprint(site_dict)

{'cyclone_track_database': {'file': ''},
 'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'gui': {'default_area_units': 'm2',
         'default_discharge_units': 'm3/s',
         'default_distance_units': 'meters',
         'default_intensity_units': 'mm/hr',
         'default_length_units': 'meters',
         'default_velocity_units': 'm/s',
         'default_volume_units': 'm3',
         'tide_harmonic_amplitude': {'units': 'meters', 'value': 0}},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'risk': {'flooding_threshold': {'units': 'meters', 'value': 0.15},
          'return_periods': [1, 2, 5, 10, 25, 50, 100]},
 'sfincs': {'ambient_air_pressure': 102005,
            'csname': 'WGS 84 / UTM zone 30N',
            'cstype': 'projected',
            'floodmap_units': 'meters',
            'off

## DEM

The high resolution DEM used for downscaling the floodmap. Here taken from the sfincs subgrid dem.

Unsure what the index file does, so double check that this is indeed the correct one.

In [12]:
if not (model_path / "static" / "dem").exists():
    makedirs(model_path / "static" / "dem")

shutil.copy((model_path / "static" / "templates" / "overland" / "subgrid" / "dep_subgrid.tif"),
            model_path / "static" / "dem" / (model_name+"_subgrid.tif"))
# Not sure this is the right thing to do. Is this the needed index file and is this the correct location?
shutil.copy((model_path / "static" / "templates" / "overland" / "sfincs.ind"),
            model_path / "static" / "dem" / (model_name+"_index.tif"))

# Site object cannot parse Path objects
dem_dict = {
    "filename": (model_path / "static" / "dem" / (model_name+"_subgrid.tif")).name,
    "indexfilename": (model_path / "static" / "dem" / (model_name+"_index.tif")).name,
    "units": "meters"
}

site_dict.update({"dem": dem_dict})
pprint(site_dict)

{'cyclone_track_database': {'file': ''},
 'dem': {'filename': 'Humber_subgrid.tif',
         'indexfilename': 'Humber_index.tif',
         'units': 'meters'},
 'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'gui': {'default_area_units': 'm2',
         'default_discharge_units': 'm3/s',
         'default_distance_units': 'meters',
         'default_intensity_units': 'mm/hr',
         'default_length_units': 'meters',
         'default_velocity_units': 'm/s',
         'default_volume_units': 'm3',
         'tide_harmonic_amplitude': {'units': 'meters', 'value': 0}},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'risk': {'flooding_threshold': {'units': 'meters', 'value': 0.15},
          'return_periods': [1, 2, 5, 10, 25, 50, 100]},
 'sfincs': {'ambient_air_pressure': 102005,
            'csname'

## Benefits model

In [13]:
benefit_dict = {
    "current_year": 2023, # Could get this from SFINCS sim times, maybe not right when running scenarios?
    "current_projection": "current", # Creating this toml file in right location done in another notebook
    "baseline_strategy": "no_measures", # Creating this toml file in right location done in another notebook
    "event_set": "test_set", # Creating this toml file in right location done in another notebook
}

site_dict.update({"benefits": benefit_dict})
pprint(site_dict)

{'benefits': {'baseline_strategy': 'no_measures',
              'current_projection': 'current',
              'current_year': 2023,
              'event_set': 'test_set'},
 'cyclone_track_database': {'file': ''},
 'dem': {'filename': 'Humber_subgrid.tif',
         'indexfilename': 'Humber_index.tif',
         'units': 'meters'},
 'description': 'Humber Delta, UK',
 'fiat': {'aggregation': [{'field_name': '', 'file': '', 'name': ''}],
          'bfe': {'field_name': '', 'geom': ''},
          'exposure_crs': 'EPSG:4326',
          'floodmap_type': 'water_level'},
 'gui': {'default_area_units': 'm2',
         'default_discharge_units': 'm3/s',
         'default_distance_units': 'meters',
         'default_intensity_units': 'mm/hr',
         'default_length_units': 'meters',
         'default_velocity_units': 'm/s',
         'default_volume_units': 'm3',
         'tide_harmonic_amplitude': {'units': 'meters', 'value': 0}},
 'lat': 53.66,
 'lon': -0.47,
 'name': 'Humber',
 'risk': {'flood

## River
River is (should be?) optional, but not specifying anything will create a None type when reading in site.toml, which doesn't play well with scenario.run(). The underlying problem is in event.add_dis_ts() where len(site_river) is used. So river should at least be an emtpy list

In [14]:
site_dict.update({"river": []})

## Save file

Write the created site dictionary to site.toml at the specified location.

In [15]:
with open(site_fn,"wb") as f:
    tomli_w.dump(site_dict,f)

## Test load

In [16]:
#test_load = Site.load_file(Path('n:/Deltabox/Postbox/Tromp, Willem/Database_env_fix/static/site/site.toml'))
test_load = Site.load_file(site_fn)

In [17]:
vars(test_load.attrs)

{'name': 'Humber',
 'description': 'Humber Delta, UK',
 'lat': 53.66,
 'lon': -0.48,
 'sfincs': SfincsModel(csname='WGS 84 / UTM zone 30N', cstype=<Cstype.projected: 'projected'>, version='SFINCS_v2.0.1_tag_exe', offshore_model='', overland_model='overland', ambient_air_pressure=102005.0, floodmap_units=<UnitTypesLength.meters: 'meters'>, save_simulation=False),
 'water_level': WaterLevelReferenceModel(reference=VerticalReferenceModel(name='MSL', height=UnitfulLength(value=0.0, units=<UnitTypesLength.meters: 'meters'>)), localdatum=VerticalReferenceModel(name='EGM2008', height=UnitfulLength(value=0.0, units=<UnitTypesLength.meters: 'meters'>)), msl=VerticalReferenceModel(name='MSL', height=UnitfulLength(value=0.0, units=<UnitTypesLength.meters: 'meters'>)), other=None),
 'cyclone_track_database': Cyclone_track_databaseModel(file=''),
 'slr': SlrModel(vertical_offset=UnitfulLength(value=0.2, units=<UnitTypesLength.meters: 'meters'>), relative_to_year=2020),
 'gui': GuiModel(tide_harmoni

## River, SCS, Obs station (Optional)

The below settings are optional, they are not strictly needed to run FloodAdapt. Uncomment, change the relevant settings you want to add, and save the site.toml file again if you want to include them.

In [ ]:
# if not (model_path / "static" / "scs").exists():
#     shutil.copytree((model_path.parents[1] / "data" / "rainfall_series"),(model_path / "static" / "scs"))

# site_dict.update({"river": []})
# site_dict.update({"scs": {
#     "file": "",
#     "type": ""
# }})

# obs_dict = {
#     "name": "",
#     "ID": -9999,
#     "lat": np.nan,
#     "lon": np.nan,
#     "mhhw": {
#         "value": np.nan,
#         "units": "meters"
#     },
#     "mllw": {
#         "value": np.nan,
#         "units": "meters"
#     },
#     "localdatum": {
#         "value": np.nan,
#         "units": "meters"
#     },
#     "msl": {
#         "value": np.nan,
#         "units": "meters"
#     }
# }

# site_dict.update({"obs_station":obs_dict})